# Imports

In [ ]:
import sys, os
is_conda = os.path.exists(os.path.join(sys.prefix, 'conda-meta'))

if not is_conda:
    import findspark 
    findspark.init()

from pyspark.sql import SparkSession
# import pandas as pd
# import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pyspark.sql.functions import col, datediff, unix_timestamp
import csv
from IPython.core.display import display, HTML
from collections import defaultdict

# Para una lectura más distendida de la memoria
MODO_JAJAS = False

# Lectura de datos

In [ ]:
spark = SparkSession.builder.appName("taxis").master("local[*]").getOrCreate()
df = spark.read.csv('./tripdata_2017_01.csv', header=True, inferSchema=True)

In [ ]:
df.printSchema()
dfP=df.toPandas()

# Limpieza de datos

In [ ]:
display(dfP)
display(dfP.describe().T)

### Elementos extraños en el dataset

Lista de comportamientos extraños en los datos, y por tanto, inválidos a la hora de utilizar datos que deberían ser coherentes basándonos en la información de cada campo proporcionada por la [documentación](https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf)

* Existen carreras en las que la distancia es 0
* Existen propinas negativas
* "extra" con valores diferentes a 0 (ya que puede no haber extras), 0.5 y 1
* Existen viajes con un precio final negativo
* "MTA_tax" debe valer siempre 0.50. Valores diferentes son erróneos, y por tanto puede que el resto de la información también
    * De forma similar, "Improvement_surcharge" no debe valer menos de 0.30
* Carreras cuya fecha de fin sea igual o anterior a la fecha de inicio
* Existen tarifas con valores negativos. No tiene sentido ya que la tarifa va en función del tiempo y la distancia recorridas
* "Improvement_surcharge" es un valor en desuso, por lo que debería valer en el menor caso 0, no -0.3

### Elementos extraños PERO posibles

* Número de pasajeros es 0. Dado que es un valor que introduce el propio conductor, muy probablemente le de bastante igual introducir bien el valor.
* Un viaje empieza y acaba en la misma zona.



### Limpieza realizada

A partir de los comportamientos observados se ha procedido a eliminar las carreras que cumplen las siguientes condiciones:

- Campo "tip_amount" con valores menor a 0
- Campo "total_amount" con valores menor o igual a 0
- Campo "trip_distance" con valores menor o igual a 0
- Campo "fare_amount" con valores menor o igual a 0
- Campo "extra" con valores diferentes de 0, 0.5 y 1
- Campo "MTA_tax" con valor distinto de 0.5
- Campo "Improvement_surcharge" con valor distinto de 0 o 0.3
- Campo "tpep_dropoff_datetime" es anterior o igual a "tpep_pickup_datetime"

In [ ]:
# Convertimos las fechas a timestamp, para que dejen de ser strings a secas
# y guardamos su diferencia para luego tener más fácil el filtrado y otros cálculos

# ----------------------------------------------------------------------------------
# ESTO ES ABSURDAMENTE LENTO, TIENE QUE HABER ALGUNA FORMA MÁS FÁCIL DE HACER ESTO
# ----------------------------------------------------------------------------------
df = df.withColumn(
    "tpep_pickup_timestamp", unix_timestamp(col("tpep_pickup_datetime").cast("timestamp"))
).withColumn(
    "tpep_dropoff_timestamp", unix_timestamp(col("tpep_dropoff_datetime").cast("timestamp"))
).withColumn(
    "time_diff", col("tpep_dropoff_timestamp") - col("tpep_pickup_timestamp")  # Segundos
)

df.createOrReplaceTempView('datosCarreras')
# display(df.toPandas())

In [ ]:
datosLimpios = spark.sql("""
    SELECT * FROM datosCarreras WHERE
        tip_amount >= 0 AND
        total_amount > 0 AND
        trip_distance > 0 AND
        fare_amount > 0 AND
        (extra == 0 OR extra == 0.5 OR extra == 1) AND
        mta_tax == 0.5 AND
        improvement_surcharge >= 0 AND
        time_diff > 0
""")
print(datosLimpios.count())
datosLimpios.createOrReplaceTempView('datosCarrerasLimpios')
datosLimpiosP = datosLimpios.toPandas()

In [ ]:
display(datosLimpiosP)
display(datosLimpiosP.describe().T)

## Extracción de información

Ahora que ya hemos limpiado los datos y tenemos entradas coherentes, se puede proceder a extraer información de los mismos. 

La información que se va a extraer es:

* Velocidad media de los taxis en función de la hora.
* Viajes en taxi más comunes
* Registros financieros (propinas, personas, etc.)
    * Timos a turistas
    * Propinas en función de la hora
    * Identificar pasajeros borrachos
* Zonas con poca cobertura



In [ ]:
if not os.path.exists("taxi+_zone_lookup.csv"):
    !wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

In [ ]:
# Diccionario para traducir el id de las localizaciones a cosas coherentes
# TODO CAMBIAR PARA HACERLO CON SPARK Y UN JOIN
zone_lookup_csv = csv.DictReader(open("taxi+_zone_lookup.csv"), delimiter=",")
zone_lookup = {l['LocationID']: {"distrito": l['Borough'], "zona": l['Zone'], "servicio": l['service_zone']} for l in zone_lookup_csv}

### Velocidad media de los taxis

En este apartado se realizará un análisis de la velocidad media de los taxis, para ello se realizará una transformación de millas a metros sabiendo que 1 milla = 1609.344 metros luego dividiéndolo entre la diferencia de tiempo calculada previamente.

In [ ]:
dfMTS = datosLimpios.withColumn(
    "mean_speed", col("trip_distance")*1609.344/col("time_diff")
)

In [ ]:
dfMTSP = dfMTS.toPandas()
display(dfMTSP.sort_values(by=["mean_speed"],ascending=False).head(50))
display(dfMTSP.describe().T)

En vista de que las velocidades medias estaban mal y esto, como se puede ver en la tabla, es debido a que el time_diff es muy bajo, probablemente por un error de los tiempos almacenador por los taxistas, por lo tanto se volverá a realizar una consulta eliminando tiempos menores a 3 minutos y se comprobará las velocidades promedio otra vez.


In [ ]:
datosLimpiosSinVelocidades = spark.sql("SELECT * FROM datosCarrerasLimpios where time_diff >= 180")
dfMTS = datosLimpiosSinVelocidades.withColumn(
    "mean_speed", col("trip_distance")*1609.344/col("time_diff")
)

In [ ]:
resultsTolls = spark.sql("SELECT * FROM datosCarrerasLimpios where tolls_amount > 100").toPandas()
display(resultsTolls)
display(resultsTolls.describe().T)

In [ ]:
resultsTimos = spark.sql("SELECT * FROM datosCarrerasLimpios where PULocationID == DOLocationID").toPandas()

In [ ]:
dfMTSP = dfMTS.toPandas()
display(dfMTSP.sort_values(by=["mean_speed"],ascending=False).head(50))
display(dfMTSP.describe().T)


### [referenica a velocidades medias]

Como se puede observar, la mayoría de velocidades entre las 50 más rápidas superan el límite de velocidad nacional para zonas de carretera (24.72222 m/s) siendo que solo los 5 últimos lo cumplen, o en otras palabras que los 45 primeros infringen la ley.

Por otro lado se puede ver que los 8 primeros tienen velocidades mayores a 52 metros por segundo, lo que implica velocidades de 187.2 km/s esto puede ser debido a que haya algún tipo de fallo en el tiempo o que lleve velocidades demasiado altas.

Por último mencionar que los 6 primeros tienen velocidades mayores a 65 m/s, cosa que ya debe ser debido a un fallo, accidental o adrede por parte del conductor.


### Zonas de poca cobertura 100% real link megaupload

In [ ]:
if MODO_JAJAS:
    display(HTML('<iframe src="https://giphy.com/embed/PmdOx0iRRtqkBFlEgI" width="240" height="240" frameBorder="0" class="giphy-embed" allowFullScreen></iframe>'))

In [ ]:
sinCobertura = spark.sql("""
    SELECT DOLocationID as locationID, store_and_fwd_flag
      FROM datosCarrerasLimpios
      WHERE store_and_fwd_flag == 'Y'
""")

sC_rdd = sinCobertura.rdd
# sC_rdd.flatMap(lambda x: x['locationID']).map(lambda x: (x,1))
zone_tuples = sC_rdd.map(
    lambda x: (x['locationID'],1)
).reduceByKey(
    lambda x,y: x+y
).sortBy(
    lambda x: x[1], False
)

In [ ]:
distritos = defaultdict(int)
zonas = defaultdict(int)

for i, loc_id in enumerate(zone_tuples.collect()):
    num = loc_id[1]
    loc_id = str(loc_id[0])
    zona = zone_lookup[loc_id]
    
    distritos[zona['distrito']] += num
    zonas[zona['zona']] = num

distritos_x = [k for k in distritos.keys()]
distritos_y = [v for v in distritos.values()]
zonas_x = [k for k in zonas.keys()]
zonas_y = [v for v in zonas.values()]


fig = plt.figure(figsize=(30, 10))

ax = fig.add_subplot(1, 2, 1)
ax.bar(distritos_x, distritos_y)
ax.set_xlabel("Zona")
ax.set_ylabel("Registros guardados")
ax.set_title("Registros guardados por zona")
plt.xticks(rotation=90)

ax = fig.add_subplot(1, 2, 2)
ax.bar(zonas_x[:10], zonas_y[:10])
ax.set_xlabel("Zona")
ax.set_ylabel("Registros guardados")
ax.set_title("Registros guardados por zona")

plt.xticks(rotation=90)

plt.show()

# print(f"TOP {i+1} con {num}\n    Distrito: {zona['distrito']}\n    Zona: {zona['zona']}\n    Servicio: {zona['servicio']}")

### Viajes más comunes

# Fin

In [ ]:
#spark.stop()

IDEAS

propinas / hora

Timos

- Vueltas de mas en misma zona
- Tolls valores raros
- Diferencias exageradas de distancias para pares de datos con mismo origen y destino

Velocidad media de los taxis en función de la hora.

Viajes en taxi más comunes

Registros financieros (propinas, personas, etc.)

Zonas sin cobertura a partir del parámetro Store_and_fwd_flag

Fare_amount frente a time_diff y trip distance, infracciones de ley
